In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'flax-community/papuGaPT2'
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()  

In [2]:
from itertools import permutations
words = "babuleńka miała dwa rogate koziołki".split()
sorted(permutations(words), key=lambda l: sentence_prob(" ".join(l)), reverse=True)[:10]

[('miała', 'babuleńka', 'dwa', 'rogate', 'koziołki'),
 ('miała', 'babuleńka', 'dwa', 'koziołki', 'rogate'),
 ('dwa', 'rogate', 'koziołki', 'miała', 'babuleńka'),
 ('dwa', 'babuleńka', 'miała', 'koziołki', 'rogate'),
 ('miała', 'dwa', 'koziołki', 'rogate', 'babuleńka'),
 ('miała', 'koziołki', 'babuleńka', 'dwa', 'rogate'),
 ('miała', 'dwa', 'rogate', 'koziołki', 'babuleńka'),
 ('miała', 'babuleńka', 'rogate', 'dwa', 'koziołki'),
 ('miała', 'dwa', 'koziołki', 'babuleńka', 'rogate'),
 ('dwa', 'rogate', 'koziołki', 'babuleńka', 'miała')]

In [41]:
words = set("wczoraj wieczorem spotkałem pewną wspaniałą kobietę, która z pasją opowiadała o modelach językowych".split())

def test_synergy(w: str, v: str, wv: str):
	return 0.94*sentence_prob(wv) > sentence_prob(w) + sentence_prob(v)

def synergize(words: set):
	synergies = filter(lambda args: test_synergy(*args) and args[0] != args[1], ((w, v, f"{w} {v}") for w, v in product(words, words)))
	return max(synergies, key=lambda args: sentence_prob(args[-1]), default=None)

while (s := synergize(words)):
	w, v, wv = s
	words.remove(w)
	words.remove(v)
	words.add(wv)

print(words)
sorted(permutations(words), key=lambda l: sentence_prob(" ".join(l)), reverse=True)[:10]

{'wczoraj wieczorem', 'wspaniałą pasją', 'pewną', 'modelach spotkałem kobietę, która opowiadała o', 'językowych', 'z'}


[('językowych',
  'modelach spotkałem kobietę, która opowiadała o',
  'wczoraj wieczorem',
  'z',
  'pewną',
  'wspaniałą pasją'),
 ('językowych',
  'wczoraj wieczorem',
  'z',
  'pewną',
  'wspaniałą pasją',
  'modelach spotkałem kobietę, która opowiadała o'),
 ('wczoraj wieczorem',
  'z',
  'pewną',
  'wspaniałą pasją',
  'modelach spotkałem kobietę, która opowiadała o',
  'językowych'),
 ('wczoraj wieczorem',
  'z',
  'pewną',
  'wspaniałą pasją',
  'językowych',
  'modelach spotkałem kobietę, która opowiadała o'),
 ('językowych',
  'modelach spotkałem kobietę, która opowiadała o',
  'z',
  'pewną',
  'wspaniałą pasją',
  'wczoraj wieczorem'),
 ('modelach spotkałem kobietę, która opowiadała o',
  'wczoraj wieczorem',
  'z',
  'pewną',
  'wspaniałą pasją',
  'językowych'),
 ('modelach spotkałem kobietę, która opowiadała o',
  'językowych',
  'wczoraj wieczorem',
  'z',
  'pewną',
  'wspaniałą pasją'),
 ('językowych',
  'z',
  'pewną',
  'wspaniałą pasją',
  'wczoraj wieczorem',
  'mo